In [2]:
# coding: utf-8
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local[*]', 'pyspark')


In [50]:
import numpy
import csv
from Similarities import sim_pearson
top="33173 33475 1076 35300 15743"
train_rdd = sc.textFile("train_no_header.csv")
test_rdd = sc.textFile("test_no_header.csv")

avgVoteNumberPerItem=5.07044729863
minimumVotesRequired=avgVoteNumberPerItem*0.8
avgVote=6.81887639657

num_vote_for_item_rdd=(train_rdd
         .map(lambda x: x.split(','))
         .map(lambda x:(x[1],[x[0],int(x[2])]))
         .groupByKey()
         .map(lambda x:(x[0],len(x[1]))))
print num_vote_for_item_rdd.take(10)

def test_item_filter(x):
    for i in x[1]:
        if i[0] in test_users:
            return True
    return False

def map_string (x):
        y=x[1]
        s=""
        
        for i in y:
            s+=str(i[0])+" "
        z=len(y)
        top_rates=[33173, 33475, 1076, 35300, 15743]
        i=0
        while (z<5):
            s+=str(top_rates[i])
            i+=1
            z+=1
        if (y[0][1]<avgVote):
            s=top
        return x[0],s



def compute_rec (x):
    num=sum(i[0]*i[1] for i in x[1])
    den=sum(i[1] for i in x[1])
    if den==0:
        return 0
    R=float(num)/den
    v=(list(x[1]))[0][3]
    m=minimumVotesRequired
    C=avgVote    
    return x[0][0],(x[0][1],(v*R+m*C)/(v+m),v)

i=400
k=200
while (i<4000):
    f1= open('submission_item_based'+str(i)+'.csv','w+')
    
    print "FILE: from "+str(i)+" to"+str(i+k)
    test_users=test_rdd.collect()[i:i+k]

    test_item_votes_rdd=(train_rdd
                     .map(lambda x: x.split(','))
                     .map(lambda x:(x[1],[x[0],int(x[2])]))
                     .groupByKey()
                     .filter(test_item_filter)
                     .map(lambda x: (x[0],list(x[1]))))
                          

    #print test_item_votes_rdd.take(2)
        
    rdd = train_rdd
    rdd = rdd.map( lambda x: x.split(',') )
    rdd = rdd.map(lambda x:(x[1],[x[0],int(x[2])]))

    rdd = rdd.groupByKey()
    rdd = rdd.map(lambda x:(x[0],list(x[1]),len(x[1])))



    #computing similarities [test_voted_item,[item,sim,n_test_item_votes,n_item_votes]]
    

    sim_rdd=test_item_votes_rdd.cartesian(rdd)
    sim_rdd=sim_rdd.map(lambda x: (x[0][0],[x[1][0],sim_pearson(x[0][1],x[1][1]),len(x[0][1]),len(x[1][1])])).filter(lambda x:x[1][1]>0 and x[1][0]!=x[0])

    #print sim_rdd.take(1)
    
    #computing useful part of reverse urm 
    rev_urm_rdd=(train_rdd
         .map(lambda x: x.split(','))
         .filter(lambda x:x[0] in test_users)
         .map(lambda x:(x[1],[x[0],int(x[2])])))
    
    

    #grouping votes
    vote_rdd=rev_urm_rdd.join(sim_rdd)

    grouped_vote_rdd=(vote_rdd
                    .map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1],x[1][1][2],x[1][1][3],x[0])))
                    .groupByKey())

    #print grouped_vote_rdd.map(lambda x: (x[0],list(x[1]))).take(2)
    #computing recommendations
    rec_rdd=(grouped_vote_rdd
         .map(compute_rec)
         .groupByKey()
         .map(lambda x: (x[0],(sorted(list(y for y in x[1]),key=lambda x: -x[1]))[0:5])))
    
    

    recommendations_rdd=(rec_rdd
                     .map(map_string)
                     .takeOrdered(k+1,key=lambda x: int(x[0])))
    print recommendations_rdd            
    recommended_users=list(item[0] for item in recommendations_rdd)
    formatted_recommendations=recommendations_rdd;
    csv_f1=csv.writer(f1)
    m=0
    n=0
    while(m<k):
        if (test_users[m] in recommended_users):
            csv_f1.writerow(formatted_recommendations[n])
            n+=1
        else:
            csv_f1.writerow((test_users[m],top))
        m+=1
        
    i=i+k
    f1.close()
    



[(u'35236', 1), (u'35182', 7), (u'35234', 2), (u'35232', 1), (u'4446', 2), (u'35230', 5), (u'31739', 2), (u'35548', 2), (u'20050', 6), (u'6972', 1)]
FILE: from 400 to600
[(u'1431', '19127 17207 34930 19426 9420 '), (u'1433', '24639 33173 18200 5578 14734 '), (u'1436', '24639 31992 25694 34930 35300 '), (u'1438', '24639 33173 19127 22029 6627 '), (u'1444', '21099 6263 26804 17445 7393 '), (u'1445', '33173 33475 1076 35300 15743'), (u'1447', '33173 33475 1076 35300 15743'), (u'1452', '33173 23089 10446 35738 27660 '), (u'1453', '14734 18741 3077 33788 15452 '), (u'1463', '28199 25694 17207 9420 1289 '), (u'1470', '24639 33562 19127 25694 19426 '), (u'1473', '33173 33475 1076 35300 15743'), (u'1474', '24639 31992 33562 19127 22029 '), (u'1476', '33562 28199 25694 6627 17207 '), (u'1481', '24639 33173 28199 6627 18200 '), (u'1484', '28199 6627 22029 17207 30453 '), (u'1486', '25694 18200 22029 15687 12997 '), (u'1487', '19478 3431 26951 19837 16345 '), (u'1493', '33173 33475 1076 35300 157

In [51]:
#writing last file chunk
i=4000
k=196
while (i<4001):
    f1= open('submission_item_based'+str(i)+'.csv','w+')
    
    print "FILE: from "+str(i)+" to"+str(i+k)
    test_users=test_rdd.collect()[i:i+k]

    test_item_votes_rdd=(train_rdd
                     .map(lambda x: x.split(','))
                     .map(lambda x:(x[1],[x[0],int(x[2])]))
                     .groupByKey()
                     .filter(test_item_filter)
                     .map(lambda x: (x[0],list(x[1]))))
                          

    #print test_item_votes_rdd.take(2)
        
    rdd = train_rdd
    rdd = rdd.map( lambda x: x.split(',') )
    rdd = rdd.map(lambda x:(x[1],[x[0],int(x[2])]))

    rdd = rdd.groupByKey()
    rdd = rdd.map(lambda x:(x[0],list(x[1]),len(x[1])))



    #computing similarities [test_voted_item,[item,sim,n_test_item_votes,n_item_votes]]
    

    sim_rdd=test_item_votes_rdd.cartesian(rdd)
    sim_rdd=sim_rdd.map(lambda x: (x[0][0],[x[1][0],sim_pearson(x[0][1],x[1][1]),len(x[0][1]),len(x[1][1])])).filter(lambda x:x[1][1]>0 and x[1][0]!=x[0])

    #print sim_rdd.take(1)
    
    #computing useful part of reverse urm 
    rev_urm_rdd=(train_rdd
         .map(lambda x: x.split(','))
         .filter(lambda x:x[0] in test_users)
         .map(lambda x:(x[1],[x[0],int(x[2])])))
    
    

    #grouping votes
    vote_rdd=rev_urm_rdd.join(sim_rdd)

    grouped_vote_rdd=(vote_rdd
                    .map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1],x[1][1][2],x[1][1][3],x[0])))
                    .groupByKey())

    #print grouped_vote_rdd.map(lambda x: (x[0],list(x[1]))).take(2)
    #computing recommendations
    rec_rdd=(grouped_vote_rdd
         .map(compute_rec)
         .groupByKey()
         .map(lambda x: (x[0],(sorted(list(y for y in x[1]),key=lambda x: -x[1]))[0:5])))
    
    

    recommendations_rdd=(rec_rdd
                     .map(map_string)
                     .takeOrdered(k+1,key=lambda x: int(x[0])))
    print recommendations_rdd            
    recommended_users=list(item[0] for item in recommendations_rdd)
    formatted_recommendations=recommendations_rdd;
    csv_f1=csv.writer(f1)
    m=0
    n=0
    while(m<k):
        if (test_users[m] in recommended_users):
            csv_f1.writerow(formatted_recommendations[n])
            n+=1
        else:
            csv_f1.writerow((test_users[m],top))
        m+=1
        
    i=i+k
    f1.close()
    




FILE: from 4000 to4196
[(u'14703', '33173 33475 1076 35300 15743'), (u'14708', '25920 30087 28688 12354 9893 '), (u'14709', '34930 20620 9420 13367 10059 '), (u'14715', '30616 26197 31220 31136 33780 '), (u'14716', '33173 33475 1076 35300 15743'), (u'14718', '33173 33475 1076 35300 15743'), (u'14720', '6699 17082 11868 24109 21651 '), (u'14723', '15687 30360 34351 23507 26197 '), (u'14726', '33532 35306 24178 6458 7911 '), (u'14727', '28199 30360 26197 6111 19725 '), (u'14728', '162 4469 17853 21298 36583 '), (u'14730', '33603 20941 15973 32083 1149 '), (u'14733', '33173 33475 1076 35300 15743'), (u'14737', '33173 22029 30453 15687 5578 '), (u'14738', '31992 35300 25920 30360 1397 '), (u'14740', '24639 31992 33562 33173 22029 '), (u'14749', '19426 20620 9420 10059 30360 '), (u'14750', '25920 6446 26804 3680 29221 '), (u'14751', '19426 14734 26743 34351 21913 '), (u'14752', '18200 17207 35300 2464 23089 '), (u'14756', '6627 19426 15687 20620 35300 '), (u'14757', '33532 9324 18142 36694 

In [52]:
#collecting results
i=0
rows=[["userId","testItems"]]
while(i<4001):
    f1= open('submission_item_based'+str(i)+'.csv','rb')
    f1_csv=csv.reader(f1)
    rows=rows+list(f1_csv)
    i=i+200
    f1.close()
f1=open('submission_item_based.csv','w+')
f1_csv=csv.writer(f1)
f1_csv.writerows(rows)
f1.close()